In [1]:
import sqlite3

pat_icd_db = '/mnt/isilon/wang_lab/mengge/project/clarity_sql/claitydata/pat_icd.db'
icd_con = sqlite3.connect(pat_icd_db)
icd_cur = icd_con.cursor()

### DS pat ids

In [13]:
import os
ds_pat = set()
ds_data_dir = '/mnt/isilon/wang_lab/mengge/project/DownSymdrome/ds_notes'
for i in os.listdir(ds_data_dir ):
    ds_pat.add(i)
    
len(ds_pat)

4406

### find what DS pats have 22q ICD codes

In [14]:
icd_22q=['Q93.81','D82.1', '279.11','758.32']


In [15]:
dspat_22q_dict = dict()

In [16]:
for icd in icd_22q:
    cmd = "select distinct(pat_id) from ICD where icd like '{}%'".format(icd)
    print('querying for {}'.format(icd))
    icd_cur.execute(cmd)
    data = icd_cur.fetchall()
    
    for d in data:
        if(d[0] in ds_pat): dspat_22q_dict[d[0]] = True
        
for p in ds_pat:
    if(dspat_22q_dict.get(p) == None): dspat_22q_dict[p] = False

querying for Q93.81
querying for D82.1
querying for 279.11
querying for 758.32


In [17]:
num_22q = 0

for pat in dspat_22q_dict.keys():
    if(dspat_22q_dict[pat] == True):
        num_22q +=1
        
print('{} of the DS patients have 22q ICD codes'.format(str(num_22q)))

115 of the DS patients have 22q ICD codes


### Query patients having heart disease ICD

In [2]:
heart_icd = [str(i) for i in range(390, 460)]
heart_icd.append('I')

In [3]:
control_heart_pat_dict = dict()

In [4]:
all_heart_pat = set()

In [7]:
all_heart_pat = set()
for icd in heart_icd:
    cmd = "select distinct(pat_id) from ICD where icd like '{}%'".format(icd)
    print('querying for ICD code: {}'.format(icd))
    icd_cur.execute(cmd)
    data = icd_cur.fetchall()
    
    for d in data:
        all_heart_pat.add(d[0])

querying for ICD code: 390
querying for ICD code: 391
querying for ICD code: 392
querying for ICD code: 393
querying for ICD code: 394
querying for ICD code: 395
querying for ICD code: 396
querying for ICD code: 397
querying for ICD code: 398
querying for ICD code: 399
querying for ICD code: 400
querying for ICD code: 401
querying for ICD code: 402
querying for ICD code: 403
querying for ICD code: 404
querying for ICD code: 405
querying for ICD code: 406
querying for ICD code: 407
querying for ICD code: 408
querying for ICD code: 409
querying for ICD code: 410
querying for ICD code: 411
querying for ICD code: 412
querying for ICD code: 413
querying for ICD code: 414
querying for ICD code: 415
querying for ICD code: 416
querying for ICD code: 417
querying for ICD code: 418
querying for ICD code: 419
querying for ICD code: 420
querying for ICD code: 421
querying for ICD code: 422
querying for ICD code: 423
querying for ICD code: 424
querying for ICD code: 425
querying for ICD code: 426
q

### Get Control pats

In [8]:
import sqlite3

db = '/mnt/isilon/wang_lab/mengge/project/autism_notes/nonpsy_nonasd_patient.db'
note_con = sqlite3.connect(db)
note_cur = note_con.cursor()

In [18]:
cmd = 'select * from patients'
note_cur.execute(cmd)

control_pats = set()


data  = note_cur.fetchone()
while(data and len(control_pats) <= (len(ds_pat) - num_22q + 2000)  ):
    pat = data[0]
    if(pat not in ds_pat and pat not in all_heart_pat): control_pats.add(pat)
    data  = note_cur.fetchone()
    if(len(control_pats) % 1000 == 0):
        print('Find {} desired control patients'.format(str(len(control_pats))))

Find 1000 desired control patients
Find 2000 desired control patients
Find 3000 desired control patients
Find 4000 desired control patients
Find 5000 desired control patients
Find 6000 desired control patients


In [20]:
with open('../cotrol_pat', 'w+') as fw:
    fw.write('\n'.join(control_pats))

### Query the Control Patients Notes Using Respublica (qsub)

In [ ]:
n_process = 40
for i in range(n_process):
    database = db
    idx = i
    num = n_process
    l = '../cotrol_pat'
    o = nonds_nonpsy_notes
    
    cmd = "echo 'python ../query_notes.py -db {} -i {} -n {} -l {} -o {} -a' | qsub -cwd -V -o ../newadded/{}.o -e ../newadded/{}.e".format(database, str(i), str(n_process), l,o, str(i),str(i))
    
    os.system(cmd)

In [21]:
icd_cur.close()
icd_con.close()

In [22]:
note_cur.close()
note_con.close()